In [1]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import KFold
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score


%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}
pd.options.mode.chained_assignment = None  # default='warn'?

# data_key DataFrame
data_key = pd.read_csv('key.csv')

# data_train DataFrame
data_train = pd.read_csv('train.csv')

# data_weather DataFrame
data_weather = pd.read_csv('weather.csv')

rain_text = ['FC', 'TS', 'GR', 'RA', 'DZ', 'SN', 'SG', 'GS', 'PL', 'IC', 'FG', 'BR', 'UP', 'FG+']
other_text = ['HZ', 'FU', 'VA', 'DU', 'DS', 'PO', 'SA', 'SS', 'PY', 'SQ', 'DR', 'SH', 'FZ', 'MI', 'PR', 'BC', 'BL', 'VC' ]

data_weather['codesum'].replace("+", "")
a = []
for i in range(len(data_weather['codesum'])):
    a.append(data_weather['codesum'].values[i].split(" "))
    for i_text in a[i]:
        if len(i_text) == 4:
            a[i].append(i_text[:2])
            a[i].append(i_text[2:])
            
data_weather["nothing"] = 1
data_weather["rain"] = 0
data_weather["other"] = 0
b = -1
for ls in a:
    b += 1
    for text in ls:
        if text in rain_text:
            data_weather.loc[b, 'rain'] = 1
            data_weather.loc[b, 'nothing'] = 0
        elif text in other_text:
            data_weather.loc[b,'other'] = 1
            data_weather.loc[b, 'nothing'] = 0      
# 모든 데이터 Merge
df = pd.merge(data_weather, data_key)

station_nbr = df['station_nbr']
df.drop('station_nbr', axis=1, inplace=True)
df['station_nbr'] = station_nbr

df = pd.merge(df, data_train)

# T 값 처리 하기. Remained Subject = > 'M' and '-'
df['snowfall'][df['snowfall'] == '  T'] = 0.05
df['preciptotal'][df['preciptotal'] == '  T'] = 0.005

# 주말과 주중 구분 작업 하기
df['date'] = pd.to_datetime(df['date'])
df['week7'] = df['date'].dt.dayofweek
df['weekend'] = 0
df.loc[df['week7'] == 5, 'weekend'] = 1
df.loc[df['week7'] == 6, 'weekend'] = 1

df1 = df[df['station_nbr'] == 1]; df11 = df[df['station_nbr'] == 11]
df2 = df[df['station_nbr'] == 2]; df12 = df[df['station_nbr'] == 12]
df3 = df[df['station_nbr'] == 3]; df13 = df[df['station_nbr'] == 13]
df4 = df[df['station_nbr'] == 4]; df14 = df[df['station_nbr'] == 14]
df5 = df[df['station_nbr'] == 5]; df15 = df[df['station_nbr'] == 15]
df6 = df[df['station_nbr'] == 6]; df16 = df[df['station_nbr'] == 16]
df7 = df[df['station_nbr'] == 7]; df17 = df[df['station_nbr'] == 17]
df8 = df[df['station_nbr'] == 8]; df18 = df[df['station_nbr'] == 18]
df9 = df[df['station_nbr'] == 9]; df19 = df[df['station_nbr'] == 19]
df10 = df[df['station_nbr'] == 10]; df20 = df[df['station_nbr'] == 20]

In [4]:
df7 = df7.apply(pd.to_numeric, errors = 'coerce')
df7.describe().iloc[:, 14:]
# 없는 Column = codesum, station_nbr, date, depart, sunrise, sunset, 

,stnpressure,sealevel,resultspeed,resultdir,avgspeed,nothing,rain,other,store_nbr,station_nbr,item_nbr,units,week7,weekend
count,298035.000000,286047.000000,297702.000000,297702.000000,297702.000000,298368.000000,298368.000000,298368.000000,298368.000000,298368.0,298368.000000,298368.000000,298368.000000,298368.000000
mean,28.637207,30.003003,9.958725,18.371365,11.815660,0.549107,0.358259,0.308036,17.000000,7.0,56.000000,1.017981,3.008929,0.287946
std,0.178843,0.200966,5.471751,9.368480,4.883206,0.497583,0.479490,0.461682,10.424348,0.0,32.041693,8.671075,2.003886,0.452807
min,28.040000,29.380000,0.200000,1.000000,1.000000,0.000000,0.000000,0.000000,3.000000,7.0,1.000000,0.000000,0.000000,0.000000
25%,28.510000,29.870000,5.800000,15.000000,8.000000,0.000000,0.000000,0.000000,3.000000,7.0,28.000000,0.000000,1.000000,0.000000
50%,28.630000,29.970000,9.000000,18.000000,10.900000,1.000000,0.000000,0.000000,20.000000,7.0,56.000000,0.000000,3.000000,0.000000
75%,28.740000,30.130000,13.600000,21.000000,15.100000,1.000000,1.000000,1.000000,28.000000,7.0,84.000000,0.000000,5.000000,1.000000
max,29.310000,30.710000,28.400000,36.000000,28.700000,1.000000,1.000000,1.000000,28.000000,7.0,111.000000,430.000000,6.000000,1.000000


In [2]:
df7['store_nbr'].unique()

array([ 3, 20, 28], dtype=int64)

In [5]:
df7_drop_columns = ['codesum', 'station_nbr', 'date', 'depart', 'sunrise', 'sunset']
df7 = df7.drop(columns = df7_drop_columns)

In [6]:
# np.nan를 포함하고 있는 변수(column)를 찾아서, 그 변수에 mean 값 대입해서 Frame의 모든 Value가 fill 되게 하기.
df7_columns = df7.columns

# Cateogry 값을 포함하는 변수는 np.nan에 mode값으로 대체하고, 나머지 실수 값을 포함한 변수는 np.nan에 mean값으로 대체
for i in df7_columns:
    if (i == 'resultdir'):
        df7[i].fillna(df7[i].mode()[0], inplace=True)
        print(df7[i].mode()[0])
    else:
        df7[i].fillna(df7[i].mean(), inplace=True)

# 이제 모든 변수가 숫자로 표기 되었기 때문에, 가능 함. 
# 상대 습도 추가 #
df7['relative_humility'] = 100*(np.exp((17.625*((df7['dewpoint']-32)/1.8))/(243.04+((df7['dewpoint']-32)/1.8)))/np.exp((17.625*((df7['tavg']-32)/1.8))/(243.04+((df7['tavg']-32)/1.8))))

# 체감온도 계산
df7["windchill"] = 35.74 + 0.6215*df7["tavg"] - 35.75*(df7["avgspeed"]**0.16) + 0.4275*df7["tavg"]*(df7["avgspeed"]**0.16)

df7 = df7[df7['units'] != 0]

18.0


In [7]:
model_df7 = sm.OLS.from_formula('np.log1p(units) ~ tmax + tmin + tavg + dewpoint + heat + cool + preciptotal + \
sealevel + resultspeed + resultdir + avgspeed + C(nothing) + C(rain) + C(other) + C(item_nbr) + C(week7) + \
C(weekend) + relative_humility + windchill + 0', data = df7)
result_df7 = model_df7.fit()

print(result_df7.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.844
Model:                            OLS   Adj. R-squared:                  0.843
Method:                 Least Squares   F-statistic:                     1309.
Date:                Wed, 04 Jul 2018   Prob (F-statistic):               0.00
Time:                        14:04:43   Log-Likelihood:                -5776.4
No. Observations:                7529   AIC:                         1.162e+04
Df Residuals:                    7497   BIC:                         1.184e+04
Df Model:                          31                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(nothing)[0]         -0.0006      0

In [8]:
model_df7 = sm.OLS.from_formula('np.log1p(units) ~ scale(tmax) + scale(tmin) + scale(tavg) + scale(dewpoint) + scale(heat) + scale(cool) + scale(preciptotal) + \
scale(sealevel) + scale(resultspeed) + scale(resultdir) + scale(avgspeed) + C(nothing) + C(rain) + C(other) + C(item_nbr) + C(week7) + \
C(weekend) + scale(relative_humility) + scale(windchill) + 0', data = df7)
result_df7 = model_df7.fit()

print(result_df7.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.844
Model:                            OLS   Adj. R-squared:                  0.843
Method:                 Least Squares   F-statistic:                     1309.
Date:                Wed, 04 Jul 2018   Prob (F-statistic):               0.00
Time:                        14:05:00   Log-Likelihood:                -5776.4
No. Observations:                7529   AIC:                         1.162e+04
Df Residuals:                    7497   BIC:                         1.184e+04
Df Model:                          31                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
C(nothing)[0]           

In [9]:
anova_result_df7 = sm.stats.anova_lm(result_df7).sort_values(by=['PR(>F)'], ascending = False)
anova_result_df7[anova_result_df7['PR(>F)'] <= 0.05]

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


,df,sum_sq,mean_sq,F,PR(>F)
scale(tmin),1.0,1.083204,1.083204,3.971310,4.631795e-02
scale(resultdir),1.0,1.105187,1.105187,4.051907,4.415717e-02
scale(preciptotal),1.0,1.950950,1.950950,7.152697,7.501372e-03
scale(heat),1.0,2.115624,2.115624,7.756436,5.365577e-03
scale(tmax),1.0,5.361734,5.361734,19.657530,9.395757e-06
C(rain),1.0,8.396120,8.396120,30.782388,2.984893e-08
C(week7),6.0,80.889600,13.481600,49.427098,7.194228e-60
C(item_nbr),10.0,10955.935458,1095.593546,4016.734657,0.000000e+00
C(nothing),2.0,72866.443463,36433.221732,133573.792001,0.000000e+00


In [10]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(df7.values, i) for i in range(df7.shape[1])]
vif["features"] = df7.columns
vif = vif.sort_values("VIF Factor").reset_index(drop=True)
vif
# 10순위까지 겹치는것만 쓴다
# resultdir, preciptotal, C(rain), C(week7), C(item_nbr)

,VIF Factor,features
0,1.099038,store_nbr
1,1.133062,snowfall
2,1.146733,resultdir
3,1.163305,item_nbr
4,1.191278,units
5,1.193851,preciptotal
6,2.595666,other
7,2.763025,week7
8,2.780916,weekend
9,4.359832,rain


In [ ]:
# resultdir, preciptotal, C(rain), C(week7), C(item_nbr), resultdir, 
model_df7 = sm.OLS.from_formula('np.log1p(units) ~ C(item_nbr) + scale(preciptotal) + scale(resultdir) + \
C(week7) + rain +  0', data = df7)
result_df7 = model_df7.fit()

print(result_df7.summary())

In [ ]:
# resultdir, preciptotal, C(rain), C(week7), item_nbr, weekend
X7 = df7[['week7', 'item_nbr', 'preciptotal', 'resultdir', 'rain']]
y7 = df7['units']
model7 = LinearRegression()

cv7 = KFold(n_splits=10, shuffle=True, random_state=0)

cross_val_score(model7, X7, y7, scoring="r2", cv=cv7)